In [2]:
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from jaxtyping import Float, Int

In [4]:
@dataclass
class GPTConfig:
	# default test values -- too small for a real language model, but big enough for testing
	d_vocab: int = 10_000
	d_model: int = 128
	d_mlp: int = 512
	n_heads: int = 4
	d_head: int = 32
	n_layers: int = 6
	act_fn: type[nn.Module] = nn.ReLU

	@property
	def n_params(self) -> int:
		"an estimate of the number of parameters"
		return (
			self.d_vocab * self.d_model # embeddings (and tied unembeddings)
			+ (
				self.d_model * self.d_mlp * 2 # mlp weights
				+ self.d_model + self.d_mlp # mlp bias
				+ self.n_heads * ( # number of heads
					4 * self.d_model * self.d_head # 4 because Q, K, O, V
				)
			) * self.n_layers, # for each layer
		)

# note: the residual stream is `n_context` by `d_model`

# this is the row-wise (last dimension) softmax of x
# F.softmax(x, dim=-1)

class AttentionHead(nn.Module):
	
	def __init__(self, cfg: GPTConfig):
		super().__init__()
		self.W_q = torch.nn.Parameter(torch.rand(cfg.d_model, cfg.d_head))
		self.W_kT = torch.nn.Parameter(torch.rand(cfg.d_head, cfg.d_model))
		self.W_o = torch.nn.Parameter(torch.rand(cfg.d_model, cfg.d_head))
		self.W_vT = torch.nn.Parameter(torch.rand(cfg.d_head, cfg.d_model))
	
	def masking_matrix (self, n_context: Int) -> Float[torch.Tensor, "n_context n_context"]:
		m = torch.full((n_context, n_context), -torch.inf)
		return torch.triu(m, diagonal=1)

	def forward(self, x: Int[torch.Tensor, "n_context d_model"]) -> Float[torch.Tensor, "n_context d_model"]:
		# Compute attention pattern
		pattern = x @ self.W_q @ self.W_kT @ x.T
		pattern += self.masking_matrix(x.size()[0])
		pattern = torch.nn.functional.softmax(pattern, dim=1) @ x @ self.W_o @ self.W_vT

		return pattern

# List of heads needs to be of nn.Module type 

class MultiHeadedAttention(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		self.head_list = nn.ModuleList([AttentionHead(cfg) for _ in range(cfg.n_heads)])

	def forward(self, x: Int[torch.Tensor, "n_context d_model"]) -> Float[torch.Tensor, "n_context d_model"]:
		output = x
		for h in self.head_list:
			output += h.forward(x)
		return output


class MLP(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		self.W_m_down = torch.nn.Parameter(torch.rand(cfg.d_model, cfg.d_mlp))
		self.W_m_up = torch.nn.Parameter(torch.rand(cfg.d_mlp, cfg.d_model))
		self.B = torch.nn.Parameter(torch.rand(cfg.d_mlp, 1))
		self.act_function = cfg.act_fn()

	def forward(self, x: Int[torch.Tensor, "n_context d_model"]) -> Float[torch.Tensor, "n_context d_model"]:
		''' 
		Instead of looping over each column to add B, make matrix of size (d_mlp, n_context) where
		its columns correspond to the values of B
		'''
		B_matrix = self.B @ torch.full((1, x.shape[0]), 1.0)

		x_out = (self.W_m_up @ x.T) + B_matrix
		x_out = ((self.W_m_down @ self.act_function(x_out)).T) + x
		
		return x_out
	
class Transformer(nn.Module):

	def __init__(self, cfg: GPTConfig):
		super().__init__()
		self.embedding = nn.Embedding(cfg.d_vocab, cfg.d_model)
		self.MHA_layers = nn.ModuleList([MultiHeadedAttention(cfg) for _ in range(cfg.n_layers)])
		self.MLP_layers = nn.ModuleList([MLP(cfg) for _ in range(cfg.n_layers)])
		# Use nn.Embedding for the embedding, and CAN new linear layer OR use transpose for the unembedding
		self.out_layer = nn.Linear(cfg.d_model, cfg.d_vocab)

	def forward(self, x: Int[torch.Tensor, "n_context"]) -> Float[torch.Tensor, "n_context d_vocab"]:
		X = self.embedding(x)
		for MHA, MLP in zip(self.MHA_layers, self.MLP_layers):
			X = MHA.forward(X)
			X = MLP.forward(X)
		return self.out_layer(X)


In [5]:
test_config = GPTConfig()
test_x = torch.randint(low=0, high=test_config.d_vocab, size=(10,))
test_T1000 = Transformer(test_config)
test_y = test_T1000.forward(test_x)
print(test_x.shape)
print(test_y.shape)

torch.Size([10])
torch.Size([10, 10000])


## Some useful helper functions

In [6]:
from pathlib import Path
import requests

def get_gutenberg_book(
	id: int|None = 84,
	data_temp: Path|str = "../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path|str = "../data/gutenberg_data",
	) -> list[str]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

def create_word_to_int_mapping(texts: list[str]) -> tuple[dict[str, int], list[list[int]]]:
    # Create vocabulary (unique words)
    vocab = set()
    for text in texts:
        words = text.split()
        vocab.update(words)
    
    # Create word to int mapping
    word_to_int = {word: idx for idx, word in enumerate(sorted(vocab))}
    
    # Convert texts to sequences of integers
    int_sequences = []
    for text in texts:
        words = text.split()
        int_sequence = [word_to_int[word] for word in words]
        int_sequences.append(int_sequence)
    
    return word_to_int, int_sequences